In [ ]:
import os
import json
import pandas as pd

# Load your DataFrame here
df = pd.read_parquet('./vantage_furniture_tutorial.parquet')

# Create the directory 'public' if it doesn't exist
output_dir = './public'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Items
# Iterate through the DataFrame and write each row to a separate JSON file
for index, row in df.iterrows():
    # Convert NaN values to None
    row_cleaned = row.where(pd.notnull(row), None)
    
    # Constructing the file name
    file_name = os.path.join(output_dir, f"{row_cleaned['id']}.json")
    
    # Convert the row to a dictionary and write to a JSON file
    with open(file_name, 'w') as file:
        json.dump(row_cleaned.to_dict(), file)

In [ ]:
# Filters
# Adjusting the function to create the JSON file in the accessible directory
def create_json_file(df, column_name):
    # Generate the nice, init upper cased without underscores version of the column name
    nice_name = column_name.replace("meta_", "").replace("_", " ").title()
    slug = column_name.replace("meta_", "")
    
    # Find distinct values in the column
    distinct_values = df[column_name].dropna().unique()
    
    # Create JSON data
    json_data = [
        {
            "name": value,
            "slug": value,
            "categoryName": nice_name,
            "categorySlug": slug,
            "count": df[df[column_name] == value].shape[0]
        }
        for value in distinct_values
    ]
    
    # Write to file
    file_path = "public/" + column_name + ".json"
    with open(file_path, 'w') as file:
        json.dump(json_data, file, indent=4)
    
    return file_path

# Creating JSON files for each column with 'meta_' prefix
json_files = [create_json_file(df, column) for column in df.columns if column.startswith('meta_')]